In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from python.fluorophores import FlMoveBleach
from python.simulators import SimSequencefile

In [ ]:
sim = SimSequencefile()
fname = os.path.join(SCRIPT_DIR, "examples", "Tracking_2D.json")
fname2 = os.path.join(SCRIPT_DIR, "settings", "PSFvectorial2D.json") # use a PSF that is defined via a json file
sim.loadsequence(fname, fname2)
sim.sequence['locLimit'] = 200
sim.sequence['Itr'][3]['phtLimit'] = 20
sim.sequence['Itr'][3]['patGeoFactor'] = 0.5
sim.makepatterns()

## make diffusing, bleaching fluorophores


In [ ]:
laserpower = 0.8 
Ds = np.arange(11)*0.1+0.5
repetitions = 5
rmse = np.zeros((len(Ds),repetitions,3))
efo = np.zeros((len(Ds),repetitions))

for d in range(len(Ds)):
    for k in range(repetitions):
        sim.posgalvo = [0, 0, 0]
        sim.posEOD = [0, 0, 0]
        sim.time = 0

        fl = FlMoveBleach()
        fl.photonbudget = np.inf
        fl.brightness = 1000*laserpower
        updatetime = 0.01 # ms
        D=Ds[d]  # um^2/s
        fl.makediffusion(D, updatetime)
        sim.fluorophores = fl
        out = sim.runSequence(repetitions=1, resetfluorophores=True)
        filter=out.loc.itr==max(out.loc.itr)
        sr=sim.summarize_results(out,display=False, filter=filter)
        rmse[d,k,:] = sr.rmse
        efo[d,k] = np.mean(out.loc.efo[filter])

rmsem = np.mean(rmse,axis=1).squeeze()
indconverged = rmse<(rmse[0,0,0]*3)
fconverged =  np.sum(indconverged,axis=1).squeeze()/repetitions
rmsec=rmse
rmsec[~indconverged] = np.nan
rmsecm = np.nanmean(rmsec, axis=1)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(131)
ax.plot(Ds, np.mean(fconverged[:,:2],axis=1))
ax.set_ylabel("fraction tracked")
ax.set_xlabel('Diffusion coefficient um^2/s')

ax = fig.add_subplot(132)
ax.plot(Ds,np.mean(efo,axis=1))
ax.set_xlabel('Diffusion coefficient um^2/s')
ax.set_ylabel("efo kHz")

ax = fig.add_subplot(133)
ax.plot(Ds, np.mean(rmsecm[:,:2],axis=1))
ax.set_ylabel("RMSE of converged (nm)")
ax.set_xlabel('Diffusion coefficient um^2/s')

fig.tight_layout()